In [7]:
%reload_ext autoreload
%autoreload 2

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

import pandas as pd
from datasets import load_from_disk
from transformers import TrainingArguments
from src.model.subset_trainer import ProteinSampleSubsetTrainer
import torch
from torch import nn

In [63]:
dataset_path = "../tmp/data/train_val_GO"
dataset = load_from_disk(dataset_path)

train_dataset = dataset["train"]

In [64]:
train_dataset = train_dataset.select(range(500))
print(train_dataset)

Dataset({
    features: ['identifier', 'term', 'aspect', 'GO Name', 'GO Sentence', 'sequence', 'species', '__index_level_0__'],
    num_rows: 500
})


In [65]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.dummy_param = nn.Parameter(torch.zeros(1))

    def forward(self, *args, **kwargs):
        return {"loss": torch.tensor(0.0)} 

model = SimpleModel()

def data_collator(features):
    return features

In [80]:
train_dataset

Dataset({
    features: ['identifier', 'term', 'aspect', 'GO Name', 'GO Sentence', 'sequence', 'species', '__index_level_0__'],
    num_rows: 500
})

In [81]:
training_args = TrainingArguments(
    output_dir="./test_output",
    per_device_train_batch_size=12,
    auto_find_batch_size=True,
    num_train_epochs=4,
    logging_dir="./logs",
    logging_steps=1,
    remove_unused_columns=False,
)

trainer = ProteinSampleSubsetTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=None,  # Not testing evaluation here
    tokenizer=None,
    data_collator=data_collator,
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [88]:
trainer.get_train_dataloader()

sampled_indices [38, 83, 100, 144, 176, 190, 223, 262, 285, 295, 341, 356, 428, 482]


In [89]:
ttt = [x for x in trainer.get_train_dataloader()]

In [90]:
print(len(ttt))
print(len(ttt[1]))

42
12


In [ ]:
trainer.per_device_train_batch_size

In [ ]:
len(ttt[1])

In [ ]:
output_data = []
for epoch in range(int(training_args.num_train_epochs)):
    dataloader = trainer.get_train_dataloader()
    for batch in dataloader:
        for example in batch:
            output_data.append({
                "epoch": epoch,
                "identifier": example["identifier"],
                "GO term": example["term"]
            })
        break


In [ ]:
df = pd.DataFrame(output_data)
display(df)

In [ ]:
df['identifier'].value_counts()